In [1]:
import pandas as pd
import tensorflow as tf

url = "https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv"
path = tf.keras.utils.get_file("titanic.csv", origin=url)


df = pd.read_csv(path)



60302/60302 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [2]:
df.head(5)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.tree import DecisionTreeClassifier

In [4]:
df.sample(5)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
507,508,1,1,"Bradley, Mr. George (""George Arthur Brayton"")",male,NaN,0,0,111427,26.5500,NaN,S
585,586,1,1,"Taussig, Miss. Ruth",female,18.0,0,2,110413,79.6500,E68,S
496,497,1,1,"Eustis, Miss. Elizabeth Mussey",female,54.0,1,0,36947,78.2667,D20,C
280,281,0,3,"Duane, Mr. Frank",male,65.0,0,0,336439,7.7500,NaN,Q
850,851,0,3,"Andersson, Master. Sigvard Harald Elias",male,4.0,4,2,347082,31.2750,NaN,S


In [53]:
df.shape

(891, 11)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [7]:
df.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [8]:
df.isnull().sum()

,0
PassengerId,0
Survived,0
Pclass,0
Name,0
Sex,0
Age,177
SibSp,0
Parch,0
Ticket,0
Fare,0


In [9]:
df.drop(columns=['Cabin'],inplace=True)

In [10]:
x_train,x_test,y_train,y_test = train_test_split(df.drop(columns=['Survived']),df['Survived'],test_size=0.2,random_state =42)

In [14]:
#apply_imputation
si_age = SimpleImputer()
si_embark= SimpleImputer(strategy='most_frequent')

x_train_age = si_age.fit_transform(x_train[['Age']])
x_train_embark = si_embark.fit_transform(x_train[['Embarked']])

x_test_age = si_age.transform(x_test[['Age']])
x_test_embarked = si_embark.transform(x_test[['Embarked']])

In [16]:
#one_hot_encoding
ohe_sex = OneHotEncoder(sparse_output=False,drop='first',handle_unknown='ignore')
ohe_embark = OneHotEncoder(sparse_output=False,drop='first',handle_unknown='ignore')

x_train_sex = ohe_sex.fit_transform(x_train[['Sex']])
x_train_embark = ohe_embark.fit_transform(x_train[['Embarked']])

x_test_sex = ohe_sex.transform(x_test[['Sex']])
x_test_embarked = ohe_embark.transform(x_test[['Embarked']])

In [18]:
#with_pipelines
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.tree import DecisionTreeClassifier

In [68]:
trf1= ColumnTransformer(transformers=[
    ('impute_age',SimpleImputer(),[4]),
    ('impute_embark',SimpleImputer(strategy='most_frequent'),[9])
],remainder="passthrough")


In [69]:
trf2= ColumnTransformer(transformers=[
    ('ohe_sex',OneHotEncoder(handle_unknown='ignore', sparse_output=False),[5,9])
],remainder="passthrough")


In [70]:
trf3= ColumnTransformer(transformers=[
    ('scale',MinMaxScaler(),[4])
],remainder="passthrough")


In [60]:
x_train


,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked
331,332,1,"Partner, Mr. Austen",male,45.5,0,0,113043,28.5000,S
733,734,2,"Berriman, Mr. William John",male,23.0,0,0,28425,13.0000,S
382,383,3,"Tikkanen, Mr. Juho",male,32.0,0,0,STON/O 2. 3101293,7.9250,S
704,705,3,"Hansen, Mr. Henrik Juul",male,26.0,1,0,350025,7.8542,S
813,814,3,"Andersson, Miss. Ebba Iris Alfrida",female,6.0,4,2,347082,31.2750,S
...,...,...,...,...,...,...,...,...,...,...
106,107,3,"Salkjelsvik, Miss. Anna Kristine",female,21.0,0,0,343120,7.6500,S
270,271,1,"Cairns, Mr. Alexander",male,NaN,0,0,113798,31.0000,S
860,861,3,"Hansen, Mr. Claus Peter",male,41.0,2,0,350026,14.1083,S
435,436,1,"Carter, Miss. Lucile Polk",female,14.0,1,2,113760,120.0000,S


In [39]:

trf4=SelectKBest(score_func=chi2,k=8)


In [40]:
trf5=DecisionTreeClassifier()

In [41]:
from sklearn.pipeline import Pipeline

In [71]:
pipe = Pipeline([
    ('trgf1',trf1),
    ('trgf2',trf2),
    ('trgf3',trf3),
    ('trgf4',trf4),
    ('trgf5',trf5)
])

In [73]:
pipe.named_steps

{'trgf1': ColumnTransformer(remainder='passthrough',
                   transformers=[('impute_age', SimpleImputer(), [4]),
                                 ('impute_embark',
                                  SimpleImputer(strategy='most_frequent'),
                                  [9])]),
 'trgf2': ColumnTransformer(remainder='passthrough',
                   transformers=[('ohe_sex',
                                  OneHotEncoder(handle_unknown='ignore',
                                                sparse_output=False),
                                  [5, 9])]),
 'trgf3': ColumnTransformer(remainder='passthrough',
                   transformers=[('scale', MinMaxScaler(), [4])]),
 'trgf4': SelectKBest(k=8, score_func=<function chi2 at 0x7ad0a690ed40>),
 'trgf5': DecisionTreeClassifier()}

In [72]:
pipe.fit(x_train,y_train)

ValueError: could not convert string to float: 'S'